# http.server - Base Classes for Implementing Web Servers

Purpose:	http.server includes classes that can form the basis of a web server.

http.server uses classes from socketserver to create base classes for making HTTP servers. HTTPServer can be used directly, but the BaseHTTPRequestHandler is intended to be extended to handle each protocol method (GET, POST, etc.).

## HTTP GET

To add support for an HTTP method in a request handler class, implement the method do_METHOD(), replacing METHOD with the name of the HTTP method (e.g., do_GET(), do_POST(), etc.). For consistency, the request handler methods take no arguments. All of the parameters for the request are parsed by BaseHTTPRequestHandler and stored as instance attributes of the request instance.

This example request handler illustrates how to return a response to the client, and some of the local attributes that can be useful in building the response.

The message text is assembled and then written to wfile, the file handle wrapping the response socket. Each response needs a response code, set via send_response(). If an error code is used (404, 501, etc.), an appropriate default error message is included in the header, or a message can be passed with the error code.

To run the request handler in a server, pass it to the constructor of HTTPServer, as in the __main__ processing portion of the sample script.

Then start the server in a seperate terminal:

$ python3 http_server_GET.py

In [1]:
! curl -i http://127.0.0.1:8080/

In [2]:
! curl -v -i http://127.0.0.1:8080/?foo=bar

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to 127.0.0.1 (127.0.0.1) port 8080 (#0)





* HTTP 1.0, assume close after body



























* Curl_http_done: called premature == 0
* Closing connection 0


#### Note
The output produced by different versions of curl may vary. If running the examples produces different output, check the version number reported by curl.

## HTTP POST

Supporting POST requests is a little more work, because the base class does not parse the form data automatically. The cgi module provides the FieldStorage class which knows how to parse the form, if it is given the correct inputs.

Run the server in one window:

$ python3 http_server_POST.py

The arguments to curl can include form data to be posted to the server by using the -F option. The last argument, -F datafile=@http_server_GET.py, posts the contents of the file http_server_GET.py to illustrate reading file data from the form.



In [3]:
! curl -v http://127.0.0.1:8081/ -F name=alvin -F foo=bar \
-F datafile=@http_server_GET.py

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to 127.0.0.1 (127.0.0.1) port 8081 (#0)
> POST / HTTP/1.1
> Host: 127.0.0.1:8081
> User-Agent: curl/7.51.0
> Accept: */*
> Content-Length: 1976
> Expect: 100-continue
> Content-Type: multipart/form-data; boundary=------------------------6350146384064063
> 
* Done waiting for 100-continue
* HTTP 1.0, assume close after body
< HTTP/1.0 200 OK
< Server: BaseHTTP/0.6 Python/3.5.1
< Date: Wed, 17 May 2017 04:49:18 GMT
< Content-Type: text/plain; charset=utf-8
< 
Client: ('127.0.0.1', 61609)
User-agent: curl/7.51.0
Path: /
Form data:
	foo=bar
	name=alvin
	Uploaded datafile as 'http_server_GET.py' (1569 bytes)
* Curl_http_done: called premature == 0
* Closing connection 0


In [4]:
# try the same command on the previous GET server.
! curl -v http://127.0.0.1:8080/ -F name=alvin -F foo=bar \
-F datafile=@http_server_GET.py

*   Trying 127.0.0.1...
* TCP_NODELAY set
* Connected to 127.0.0.1 (127.0.0.1) port 8080 (#0)








* HTTP 1.0, assume close after body







<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN"
        "http://www.w3.org/TR/html4/strict.dtd">
<html>
    <head>
        <meta http-equiv="Content-Type" content="text/html;charset=utf-8">
        <title>Error response</title>
    </head>
    <body>
        <h1>Error response</h1>
        <p>Error code: 501</p>
        <p>Message: Unsupported method ('POST').</p>
        <p>Error code explanation: HTTPStatus.NOT_IMPLEMENTED - Server does not support this operation.</p>
    </body>
</html>
* Curl_http_done: called premature == 0
* Closing connection 0


## Threading and Forking

HTTPServer is a simple subclass of socketserver.TCPServer, and does not use multiple threads or processes to handle requests. To add threading or forking, create a new class using the appropriate mix-in from socketserver.

Run the server in the same way as the other examples.

$ python3 http_server_threads.py

Each time the server receives a request, it starts a new thread or process to handle it:

In [5]:
! curl http://127.0.0.1:8083/

Thread-7


In [6]:
! curl http://127.0.0.1:8083/

Thread-8


In [7]:
! curl http://127.0.0.1:8083/

Thread-9


Swapping ForkingMixIn for ThreadingMixIn would achieve similar results, using separate processes instead of threads.

## Handling Errors

Handle errors by calling send_error(), passing the appropriate error code and an optional error message. The entire response (with headers, status code, and body) is generated automatically.

In this case, a 404 error is always returned.

$ python3 http_server_errors.py

The error message is reported to the client using an HTML document as well as the header to indicate an error code.

In [8]:
! curl -i http://127.0.0.1:8084/








<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN"
        "http://www.w3.org/TR/html4/strict.dtd">
<html>
    <head>
        <meta http-equiv="Content-Type" content="text/html;charset=utf-8">
        <title>Error response</title>
    </head>
    <body>
        <h1>Error response</h1>
        <p>Error code: 404</p>
        <p>Message: Not Found.</p>
        <p>Error code explanation: 404 - Nothing matches the given URI.</p>
    </body>
</html>


## Setting Headers

The send_header method adds header data to the HTTP response. It takes two arguments: the name of the header and the value.

This example sets the Last-Modified header to the current timestamp, formatted according to RFC 7231.

In [9]:
! curl -i http://127.0.0.1:8085/







Response body


## Command Line Use

http.server includes a built-in server for serving files from the local file system. Start it from the command line using the -m option for the Python interpreter.

The root directory of the server is the working directory where the server is started.

In [10]:
#create a dummy file
! touch index.html

In [11]:
! curl -I http://127.0.0.1:8086/index.html

In [12]:
#cleanup
!rm index.html